In [1]:
# https://runawayhorse001.github.io/LearningApacheSpark/clustering.html
from utilsom import * 

In [2]:
input_data = spark.read.csv("C:/Users/Principal/Desktop/htru.csv", header=False, inferSchema=True, mode="DROPMALFORMED", encoding='UTF-8')
#df = df.drop("_c0")
#input_data.show(n=5,truncate=False)
#http://archive.ics.uci.edu/ml/datasets/HEPMASS  
#input_data = load_data("hdfs://localhost:9000/data/htru.csv")

In [3]:
print(input_data.count())
print(input_data)

# dellet label
df_input_data = input_data.drop("_c8")

# staep 2
df_input_data = shuffling(df_input_data, 10)

# staep 3
input_features = assembler(df_input_data,df_input_data.columns[0:])

# staep 4
scaledData = scaler(input_features)

17898
DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: double, _c5: double, _c6: double, _c7: double, _c8: int]


# SOM

In [4]:
#https://machinelearningnepal.com/2018/01/22/apache-spark-implementation-of-som-batch-algorithm/
# step 5 Som
from som.batch_som import SOM
from matplotlib import pyplot as plt
from matplotlib import patches as patches

#mapsize = calculate_map_size(input_data)
#print('Som: X = {} and Y = {}, {} neurones'.format(mapsize[0], mapsize[1],mapsize[0]*mapsize[1]))

#x =  mapsize[0] 
#y = mapsize[1]
som = SOM(6,4,8) # (x×y) neurons, (70 features)
som.train(scaledData, 10) # data , epochs  +10

   # for show All codebook
#som.net
   # dimention of codebook (x,y,features)
print('dimention of codebook 3D:',som.net.shape)

#som.net.reshpae(x*y,features)
data = som.net
data = data.transpose(2,1,0).reshape(-1,data.shape[2])
print('dimention of codebook 2D:',data.shape)

SOM training epoches 1
neighborhood radius  3.4822022531844965
-------------------------------------
SOM training epoches 2
neighborhood radius  3.0314331330207964
-------------------------------------
SOM training epoches 3
neighborhood radius  2.6390158215457884
-------------------------------------
SOM training epoches 4
neighborhood radius  2.29739670999407
-------------------------------------
SOM training epoches 5
neighborhood radius  2.0
-------------------------------------
SOM training epoches 6
neighborhood radius  1.7411011265922482
-------------------------------------
SOM training epoches 7
neighborhood radius  1.5157165665103982
-------------------------------------
SOM training epoches 8
neighborhood radius  1.3195079107728944
-------------------------------------
SOM training epoches 9
neighborhood radius  1.148698354997035
-------------------------------------
SOM training epoches 10
neighborhood radius  1.0
-------------------------------------
dimention of codebook 

In [5]:
# prediction som
# pour avoir bmu, bmu, index 
pre = som.predict(scaledData)
pre.show(2)
print(pre.count())

+-----------+-----------+------------+------------+-----------+----------+-----------+-----------+--------------------+---------+--------------------+
|        _c0|        _c1|         _c2|         _c3|        _c4|       _c5|        _c6|        _c7|                 bmu|  bmu_idx|            features|
+-----------+-----------+------------+------------+-----------+----------+-----------+-----------+--------------------+---------+--------------------+
|126.1640625|51.96750447| -0.18679188|-0.146062183|1.738294314|11.0873352|12.73873147|240.5595545|[0.66014276417499...|[4.0,3.0]|[0.65499898600689...|
| 131.640625|49.76754313|-0.059808838|-0.490438395| 2.56270903|16.8883382|8.031904542|78.02820791|[0.64639010555872...|[3.0,2.0]|[0.68343135266680...|
+-----------+-----------+------------+------------+-----------+----------+-----------+-----------+--------------------+---------+--------------------+
only showing top 2 rows

17898


In [6]:
# to kmenas, select, bmu 
bmux = pre.select('bmu','bmu_idx','features')
print(bmux.show(5))

+--------------------+---------+--------------------+
|                 bmu|  bmu_idx|            features|
+--------------------+---------+--------------------+
|[0.66014276417499...|[4.0,3.0]|[0.65499898600689...|
|[0.64639010555872...|[3.0,2.0]|[0.68343135266680...|
|[0.51171312812475...|[4.0,0.0]|[0.53920097343338...|
|[0.67356597521172...|[2.0,3.0]|[0.64173595619549...|
|[0.59848563614992...|[3.0,1.0]|[0.60373149462583...|
+--------------------+---------+--------------------+
only showing top 5 rows

None


# K-means

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.

featureIndexer = VectorIndexer(inputCol="bmu", \
                               outputCol="indexedbmu",\
                               ).fit(bmux)

data = featureIndexer.transform(bmux)

In [8]:
data.show(5,True)

+--------------------+---------+--------------------+--------------------+
|                 bmu|  bmu_idx|            features|          indexedbmu|
+--------------------+---------+--------------------+--------------------+
|[0.66014276417499...|[4.0,3.0]|[0.65499898600689...|[0.66014276417499...|
|[0.64639010555872...|[3.0,2.0]|[0.68343135266680...|[0.64639010555872...|
|[0.51171312812475...|[4.0,0.0]|[0.53920097343338...|[0.51171312812475...|
|[0.67356597521172...|[2.0,3.0]|[0.64173595619549...|[0.67356597521172...|
|[0.59848563614992...|[3.0,1.0]|[0.60373149462583...|[0.59848563614992...|
+--------------------+---------+--------------------+--------------------+
only showing top 5 rows



In [9]:
from pyspark.ml.clustering import KMeans, KMeansModel

kmeans = KMeans() \
          .setK(2) \
          .setFeaturesCol("indexedbmu")\
          .setPredictionCol("cluster")

# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, kmeans])

model = pipeline.fit(bmux)

cluster = model.transform(bmux)

In [10]:
cluster = model.transform(bmux)

In [11]:
cluster.show(5)

+--------------------+---------+--------------------+--------------------+-------+
|                 bmu|  bmu_idx|            features|          indexedbmu|cluster|
+--------------------+---------+--------------------+--------------------+-------+
|[0.66014276417499...|[4.0,3.0]|[0.65499898600689...|[0.66014276417499...|      0|
|[0.64639010555872...|[3.0,2.0]|[0.68343135266680...|[0.64639010555872...|      0|
|[0.51171312812475...|[4.0,0.0]|[0.53920097343338...|[0.51171312812475...|      0|
|[0.67356597521172...|[2.0,3.0]|[0.64173595619549...|[0.67356597521172...|      0|
|[0.59848563614992...|[3.0,1.0]|[0.60373149462583...|[0.59848563614992...|      0|
+--------------------+---------+--------------------+--------------------+-------+
only showing top 5 rows



In [12]:
y_pred = cluster.select('cluster')

In [13]:
import numpy as np
y_pred = np.array(list(cluster.select('cluster').toPandas()['cluster']))

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
# load data
import names
df = pd.read_csv("htru.csv", names = names.names)
# df to values
df = df.values
# 
y_test = df[:,8]

In [15]:
# predict
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of SOM K-means = {:.6f} %".format(acc * 100))
# Matrix de confusion
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

Test Accuracy of SOM K-means = 86.657727 %
[[15421   838]
 [ 1550    89]]


              precision    recall  f1-score   support

         0.0       0.91      0.95      0.93     16259
         1.0       0.10      0.05      0.07      1639

   micro avg       0.87      0.87      0.87     17898
   macro avg       0.50      0.50      0.50     17898
weighted avg       0.83      0.87      0.85     17898

